In [1]:
import pandas as pd
df = pd.read_csv("latinlibrarystats.csv")
df["name"] = df["filename"].apply(lambda fp: fp.split("/")[-1].replace(".txt", ""))
df["ft_clusts_per_len"] = df["unique_ft_clusts"] / df["avg_sent_len"]
df = df[df["lemmas"] > 100]

In [2]:
df

,Unnamed: 0,filename,token_dupl,lemmas,lemma_dupl,avg_forms_per_lemma,avg_sent_len,unique_ft_clusts,perc_dupl_ft_clusts,name,ft_clusts_per_len
0,0,../TheLatinLibraryScraper/data/texts/agnes.txt,0.3978,11697,0.264427,1.5160,28.4599,439,0.0375,agnes,15.425212
5,5,../TheLatinLibraryScraper/data/texts/alice_ali...,0.0143,68316,0.011608,1.2346,11.4991,238,0.0035,alice_alice1,20.697272
6,6,../TheLatinLibraryScraper/data/texts/alice_ali...,0.0142,57837,0.011757,1.2074,8.5850,212,0.0037,alice_alice3,24.694234
7,7,../TheLatinLibraryScraper/data/texts/alice_ali...,0.0094,118720,0.007497,1.2697,8.6486,242,0.0020,alice_alice4,27.981407
8,8,../TheLatinLibraryScraper/data/texts/alice_ali...,0.0097,96451,0.007963,1.2305,9.0581,234,0.0024,alice_alice5,25.833232
...,...,...,...,...,...,...,...,...,...,...,...
662,662,../TheLatinLibraryScraper/data/texts/williamty...,0.5671,1536,0.468750,1.2167,18.9630,244,0.1589,williamtyre_prologus,12.867162
665,665,../TheLatinLibraryScraper/data/texts/withof_wi...,0.7609,276,0.713768,1.0711,12.5455,90,0.3261,withof_withof1,7.173887
666,666,../TheLatinLibraryScraper/data/texts/withof_wi...,0.3921,533,0.369606,1.0660,19.7407,82,0.1538,withof_withof2,4.153855
667,667,../TheLatinLibraryScraper/data/texts/withof_wi...,0.2015,412,0.199029,1.0122,10.8421,40,0.0971,withof_withof4,3.689322


In [3]:
df["token_dupl_rank"] = df.sort_values(by=["token_dupl"])["token_dupl"].rank(ascending=True)
df["lemma_dupl_rank"] = df.sort_values(by=["lemma_dupl"])["lemma_dupl"].rank(ascending=True)
df["avg_forms_per_lemma_rank"] = df.sort_values(by=["avg_forms_per_lemma"])["avg_forms_per_lemma"].rank(ascending=False)
df["avg_sent_len_rank"] = df.sort_values(by=["avg_sent_len"])["avg_sent_len"].rank(ascending=False)
df["unique_ft_clusts_rank"] = df.sort_values(by=["unique_ft_clusts"])["unique_ft_clusts"].rank(ascending=False)
df["perc_dupl_ft_clusts_rank"] = df.sort_values(by=["perc_dupl_ft_clusts"])["perc_dupl_ft_clusts"].rank(ascending=True)

mean_rank = (df["token_dupl_rank"] + df["lemma_dupl_rank"] + df["avg_forms_per_lemma_rank"] + df["avg_sent_len_rank"] + df["unique_ft_clusts_rank"] + df["unique_ft_clusts_rank"] + df["perc_dupl_ft_clusts_rank"]) / 6 
df["rank"] = mean_rank

Scale all the data for optimization later.

In [4]:
from sklearn.preprocessing import MinMaxScaler

def scale(col):
    extra_dim = [[r] for r in col]
    scaler = MinMaxScaler()
    scaler.fit(extra_dim)
    return scaler.transform(extra_dim)

to_scale = ["token_dupl", "lemma_dupl", "avg_forms_per_lemma", "avg_sent_len", "unique_ft_clusts", "perc_dupl_ft_clusts"]

df["avg_forms_per_lemma"] = -df["avg_forms_per_lemma"]
df["avg_sent_len"] = -df["avg_sent_len"]
df["unique_ft_clusts"] = -df["unique_ft_clusts"]

for c in to_scale:
    scaled = scale(list(df[c]))
    df[c] = scaled

In [5]:
df["ease_index"] = \
    (0 * df["token_dupl"]) + \
    (1 * df["lemma_dupl"]) + \
    (1 * df["avg_forms_per_lemma"]) + \
    (1 * df["avg_sent_len"]) + \
    (1 * df["unique_ft_clusts"]) + \
    (2 * df["perc_dupl_ft_clusts"])

In [6]:
df = df.sort_values(by=["ease_index"], ascending=False)
ranking = df[["name", "ease_index"]]
pd.set_option('display.max_rows', None)
ranking

,name,ease_index
137,cinna,5.466904
546,suetonius_suet.pliny,5.453541
351,miscmed,5.403344
45,balde_balde1,5.287495
549,suetonius_suet.tibullus,5.234628
200,fulbert,5.124007
471,rhetores,5.114048
350,misc,4.958155
538,suetonius_suet.crispus,4.947825
320,lotichius,4.898247


In [7]:
df = df.sort_values(by=["rank"], ascending=False)
ranking = df[["name", "rank"]]
ranking

,name,rank
350,misc,672.000000
136,christian,663.750000
566,tertullian,660.416667
343,medieval,656.083333
137,cinna,645.750000
245,index,641.000000
351,miscmed,639.333333
45,balde_balde1,637.666667
549,suetonius_suet.tibullus,627.833333
200,fulbert,621.000000
